In [1]:
%load_ext autoreload
%autoreload 2

import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['C:\\Users\\troy\\Documents\\Code\\Libraries', 'C:\\Users\\troy\\Documents\\Excel Sheets', 'C:/Users/troy/Documents/Code/Libraries'])
from datetime import date, datetime
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import autoreload

import dash_bootstrap_components as dbc
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output

from FinAPI.PortfolioAnalysis.AnalyzePortfolio import Portfolio

today = date(2022, 2, 11)
objPortfolio = Portfolio(today)

Python 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)] on win32


C:\Users\troy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (29,44,45,47,48,50,51,66,72,73,74) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


[]


## Diagnosis of CSV Read

In [20]:
trade_dump_folder = r"C:\Users\troy\Documents\Excel Sheets\Blotter Dump"
filename = "Dump_%s.csv" % today.strftime("%Y%m%d")
expected_file_path = os.path.join(trade_dump_folder, filename)
df = pd.read_csv(expected_file_path, encoding='latin1', thousands=',',
                             skipinitialspace=True)

# Parse Dates
parse_dates = ['Trade Date', 'Settlement Date', 'Maturity Date', 'AutoClose Date',
                       'Premium Date', 'Barrier1 Effective Date', 'Barrier1 Maturity Date',
                       'Barrier2 Effective Date', 'Barrier2 Maturity Date', 'First Notice Date',
                       'Upcoming Fixing Date', 'Repo Close Date']

warnings = []
for c in parse_dates:
    try:
        df[c] = pd.to_datetime(df[c])
    except Exception as e:
        warnings.append("Column %s could not be parsed to a date time column: err msg - %s" % (c, e))

# Parse Columns acccording to the specified dtype

dtypes = {'Block Id': 'Int64',
            'MarketBlockId': 'Int64',
            'Asset Type': 'str',
            'TradeID': 'Int64',
            'Trade Name': 'str',
            'Security Id': 'str',
            'Account Name': 'str',
            'Trader Name': 'str',
            'Theme Name': 'str',
            'Asset Class': 'str',
            'Entered By User Name': 'str',
            'Approved By User Name': 'str',
            'Verified By User Name': 'str',
            'Tradar Id': 'Int64',
            'Trade Date': 'str',
            'Settlement Date': 'str',
            'Upcoming Fixing Date': 'str',
            'Repo Close Date': 'str',
            'Maturity Date': 'str',
            'Is Trade Open': 'str',
            'Auto Open Close': 'str',
            'ISIN': 'str',
            'BBG Type': 'str',
            'Description': 'str',
            'Currency Code': 'str',
            'Notional': 'str',
            'Secondary CCY': 'str',
            'Secondary Notional': 'str',
            'Trade Type': 'str',
            'Trade Price': 'str',
            'Rate': 'float',
            'Counter Party': 'str',
            'Trade BBG Type': 'str',
            'Country Name': 'str',
            'Capital Allocation': 'float',
            'Trade Name Capital Perc': 'float',
            'Number Of Contracts': 'float',
            'Bond Issuer': 'str',
            'Tradar Pay Account': 'str',
            'Tradar Account': 'str',
            'Op Notes': 'str',
            'AutoClose Date': 'str',
            'FXSwap Package': 'str',
            'Barrier1': 'float',
            'Barrier1 Effective Date': 'str',
            'Barrier1 Maturity Date': 'str',
            'Barrier2': 'float',
            'Barrier2 Effective Date': 'str',
            'Barrier2 Maturity Date': 'str',
            'Strike2': 'float',
            'CuttOfTime2': 'str',
            'Receive Ccy2': 'str',
            'Pay Ccy2': 'str',
            'Option PayOut2': 'str',
            'Premium': 'float',
            'Upfront': 'float',
            'CutOffTime': 'str',
            'Option Type': 'str',
            'Option Style': 'str',
            'Option PayOut': 'str',
            'Currency 1': 'str',
            'Currency 2': 'str',
            'Currency1 Notional': 'str',
            'Currency2 Notional': 'str',
            'Upfront Ccy': 'str',
            'Premium Date': 'str',
            'Accrued Interest': 'str',
            'Fixing Trade Id': 'Int64',
            'Benchmark 1': 'str',
            'Benchmark 2': 'str',
            'Is Matured': 'bool',
            'Is NDIRS': 'bool',
            'Repo Settlement Amount': 'float',
            'First Notice Date': 'str'
        }

def manual_strip(df_col, type_val):
    df_col = df_col.str.strip()
    df_col = df_col.str.replace(',','',regex=True)
    df_col = df_col.replace( '[)]','', regex=True)
    df_col = df_col.replace('[(]','-', regex=True)
    df_col = df_col.astype(type_val)
    return df_col

for c in dtypes:
    try:
        df[c] = df[c].astype(dtypes[c])
    except Exception as e:
        try:
            df[c] = manual_strip(df[c], dtypes[c])
        except Exception as e:
            warnings.append("Column %s could not be parsed to a %s column: err msg - %s" % (c, dtypes[c], e))

warning_msg = "\n".join(warnings)
warning_msg

C:\Users\troy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3173: DtypeWarning: Columns (29,44,45,47,48,50,51,66,72,73,74) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


''

## PIvot Table Diagnosis

In [2]:
# objPortfolio.create_position_pivot_with_filters('troy', 'ALL', 'ALL', 'ALL', True, False, True)
trader = 'troy'
asset_type = 'ALL'
asset_class = 'ALL'
theme_name = 'ALL'
only_open = True
only_options = False
remove_matured = True
df = objPortfolio.get_trades_by_trader(objPortfolio.raw_data, 'troy')
print(df.shape)
if asset_type.upper() != "ALL":
    df = objPortfolio.filter_asset_type(df, asset_type)
print(df.shape)
if asset_class.upper() != "ALL":
    df = objPortfolio.filter_asset_class(df, asset_class)
print(df.shape)
if theme_name.upper() != "ALL":
    df = objPortfolio.filter_theme_names(df, theme_name)
print(df.shape)
if only_open:
    df = objPortfolio.filter_only_open_trades(df)
print(df.shape)
if only_options:
    df = objPortfolio.get_option_trades(df)
print(df.shape)
if remove_matured:
    df = objPortfolio.filter_out_matured_trades(df)
print(df.shape)

(11466, 77)
(11466, 77)
(11466, 77)
(11466, 77)
(6327, 77)
(6327, 77)
(5229, 77)


In [20]:
col = objPortfolio.raw_data['Is Trade Open'].isin(['TRUE', 'True', 'true'])

## Aggregated Swaps

In [33]:
trader_df = objPortfolio.get_trades_by_trader(objPortfolio.raw_data, trader_name='troy')
swap_df = trader_df[trader_df['Asset Type'] == 'Interest Rate Swap']

column_names = ['Trade Name', 'Block Id', 'Security Id', 'Notional', 'Currency Code', 'Trade Price', 'Upfront', 'Trade Date', 
'Settlement Date', 'Maturity Date', 'Account Name', 'Benchmark 2']

account_alloc_irs = pd.pivot_table(data=swap_df, values='Notional',
                                index=['Trade Name', 'Block Id', 'Security Id'], columns='Account Name', margins=False, 
                                           aggfunc='sum').reset_index().fillna(0)                 
account_names = [v for v in account_alloc_irs.columns if v not in ["Total", "Trade Name", "Security Id", 'Block Id']]
# Comprehensive list of swaps with notionals summed up for each blockid, so we have a list of traded swaps
agg_swap_df = swap_df[column_names].groupby(['Trade Name', 'Block Id', 'Security Id', 'Currency Code', 
                                             'Settlement Date', 'Maturity Date', 'Trade Price', 'Benchmark 2'])\
                                   .agg(dict(Notional='sum', Upfront='sum'))\
                                   .reset_index()\
                                   .sort_values(by=['Trade Name', 'Currency Code', 'Block Id', 'Settlement Date', 'Maturity Date'])
agg_swap_df = agg_swap_df.merge(account_alloc_irs)
# Group the swaps with securityID, we can aggregate notionals accross multiple trades to identify collapsed and non-collapsed swaps
agg_dict = {'Notional': 'sum', 
                                    'Block Id': 'count',
                                    'Upfront':'sum',
                                    'Settlement Date': lambda x: x.iloc[-1],
                                    'Maturity Date': lambda x: x.iloc[-1], 
                                    'Currency Code': lambda x: x.iloc[-1], 
                                    'Trade Price': lambda x: x.iloc[-1],
                                    'Benchmark 2': lambda x: x.iloc[-1]}
agg_dict.update({a:'sum' for a in account_names})
collapse_report = agg_swap_df.groupby(['Trade Name', 'Security Id'])\
                              .agg(agg_dict).reset_index()

# Remove collapsed swaps to create a list of swaps which are still alive with potential collapses
non_zero_swap_df = collapse_report[collapse_report.Notional != 0.].sort_values(by=['Trade Name', 'Currency Code']).drop('Block Id', axis=1)

# agg_swap_df2 = agg_swap_df.merge(account_alloc_irs)
# collapse_report2 = collapse_report.merge(account_alloc_irs)

final_columns = ['Trade Name', 'Security Id', 'Notional', 'Upfront', 'Settlement Date', 'Maturity Date',
                         'Currency Code', 'Trade Price', 'Benchmark 2'] + account_names



## Different types of Existing trades

In [36]:
# Split swaps into EFP and Curve Trades
efp_trades = []
delta_hedges = []
curve_trades = []
vanilla_basis_mixed_trades = []
cross_market_trades = []


for trade_name in non_zero_swap_df['Trade Name'].unique():
    asset_types = trader_df[trader_df['Trade Name']==trade_name]['Asset Type'].unique()
    if 'Futures' in asset_types:
        efp_trades.append(trade_name)
    elif 'Swaption' in asset_types:
        delta_hedges.append(trade_name)
    elif 'Basis Swap' in asset_types:
        vanilla_basis_mixed_trades.append(trade_name)
    else:
        # Single Currency or Cross Currency trades
        currency_codes = trader_df[trader_df['Trade Name']==trade_name]['Currency Code'].unique()
        if len(currency_codes)==1:
            curve_trades.append(trade_name)
        else:
            cross_market_trades.append(trade_name)

# Split the Swap Trades into separate dataframe for each categories
efp_trade_df = non_zero_swap_df[non_zero_swap_df['Trade Name'].isin(efp_trades)]
delta_hedge_df = non_zero_swap_df[non_zero_swap_df['Trade Name'].isin(delta_hedges)]
curve_trade_df = non_zero_swap_df[non_zero_swap_df['Trade Name'].isin(curve_trades)]
vanilla_basis_mixed_trade_df = non_zero_swap_df[non_zero_swap_df['Trade Name'].isin(vanilla_basis_mixed_trades)]
cross_market_trade_df = non_zero_swap_df[non_zero_swap_df['Trade Name'].isin(cross_market_trades)]

account_alloc_irs = objPortfolio.create_position_pivot(swap_df)

curve_trade_df = curve_trade_df.merge(account_alloc_irs)
vanilla_basis_mixed_trade_df = vanilla_basis_mixed_trade_df.merge(account_alloc_irs)
cross_market_trade_df = cross_market_trade_df.merge(account_alloc_irs)

data_cols = ['Trade Name', 'Block Id', 'Security Id', 'Currency Code', 'Settlement Date', 'Maturity Date', 'Trade Price', 'Benchmark 2']
cross_market_trades


['USD v EUR 15y10y DSRMTRRR MLP', 'USD v EUR 15y10y spread DSRMTR']

# ED Futures Ladder and Trades

In [8]:
from tia.bbg import LocalTerminal

In [13]:
trader_df = objPortfolio.get_trades_by_trader(objPortfolio.raw_data, trader_name='troy')
subset_df  = trader_df[trader_df['Asset Type']=='Futures']

# FF List 
tickers = ["FF%s Comdty" % i for i in range(1, 26)]
cur_static_field = ['FUT_CUR_GEN_TICKER', 'INT_RATE_FUT_START_DT', 'INT_RATE_FUT_END_DT']
snap_fields = ['PX_MID', 'PX_LAST', 'PX_YEST_CLOSE']
resp = LocalTerminal.get_reference_data(tickers, cur_static_field)
df = resp.as_frame().sort_values(by='INT_RATE_FUT_START_DT')
future_code = ["%s%s" % (v[2], d.strftime('%y')) for v, d in zip(df['FUT_CUR_GEN_TICKER'], df['INT_RATE_FUT_START_DT'])]
df = df.assign(Code=future_code)

resp = LocalTerminal.get_reference_data(tickers, snap_fields)
price_df = resp.as_frame()

snap_df = df.join(price_df)
isin_list = list(snap_df['FUT_CUR_GEN_TICKER'])

array(['FFH2K2 vs. FFK2N2 Jun Hike TRRMRR'], dtype=object)

In [14]:
# Filter Trades containing Fed fund Futures
ff_subset = subset_df[subset_df.ISIN.isin(isin_list)]
trade_names = np.unique(ff_subset['Trade Name'])
for trade_name in trade_names:
    sub_df = trader_df[trader_df['Trade Name']==trade_name]
    num_asset_types = len(np.unique(sub_df['Asset Type']))
    print(num_asset_types)

1
